# **Molecular Simulation Using Robosample**

In this lab we will simulate SARS-COV Mpro protease with Robosample using its Python interface. Then, we will run a short rigid body dynamics HMC simulation.

# Markov Chain Monte Carlo Preamble

## Monte Carlo methods
Monte carlo methods are methods that use random numbers to solve probabilistic and deterministic problems.

Let's try to estimate the value of pi by generating points in a square from a uniform distribution, using different lengths of simulation.

Please change the value of *simLen* by gradually increasing it.

<img src='https://github.com/spirilaurentiu/Notebooks/blob/main/pi.png?raw=true' />

Figure 1. Having an inscribed circle inside a square of length 2, we can randomly sample points from a 2-dim uniform distribution inside the black square and count the ones that are inside the circle. The orange area within the black square will cover $\frac{\pi}{4}$ of the black square area.


In [ ]:
# Estimation of pi
import numpy as np

# Set the length of the simulation here:
simLen = 10

# Run the simulation
inCircle = 0.0
allPoints = 0.0
for i in range(simLen):
  x = np.random.uniform()
  y = np.random.uniform()
  d = np.sqrt(x**2 + y**2)
  if d < 1:
    inCircle += 1.0
  allPoints += 1.0
myPi = (inCircle / allPoints) * 4

# Print the result
print("My pi estimation: ", myPi)

## Markov Chains

A stochastic process is a set of indexed random variables ($X_i$) which take values from a state space $S$. The transition probability from a given state A to a state B is defined as $P(A \rightarrow B) = P(X_i = A | X_{i-1} = B)$.

**Markov chains are memoryless stochastic processes.** A realization of a Markov chain is a set of values taken by the random variables once.

Markov chains cand be represented graphically as graph diagrams of states connected by edges. Edges can be weighted by transition probabilities.

Important properties of Markov chains:
1. ireducibility
2. aperiodicity
3. reversibility (garantueed by the detailed balance condition)

$\pi(A) \times P(A \rightarrow B) = \pi(B) \times P(B \rightarrow A) $

Let's draw some diagrams and check if they are well behaved (have the three required properties).

First install necessary software.

In [ ]:
# Install PyGraphviz
!if pip list | grep pygraphviz ; then echo "PyGraphviz" is installed; else apt install libgraphviz-dev && pip install pygraphviz; fi


Let's see if this Markov chain violates the properties above.
If so, please try to correct it.

In [ ]:
import pygraphviz as pgv
from IPython.display import Image
import numpy as np

# Define a graph
myGraph={'1': {'2': None},
         '2': {'2': None, '3': None},
         '3': {'2': None},
         '4': {}}
A=pgv.AGraph(myGraph, directed=True)
#A.to_string()

# Build a transition probability matrix
Q = np.zeros((A.number_of_nodes(), A.number_of_nodes()))
for edge in (A.iteredges()):
  nodes = [int(edge[0]), int(edge[1])]
  w = 1.0 / len(np.unique(A.neighbors(nodes[0])))
  Q[nodes[0] - 1, nodes[1] - 1] = w
  edge = A.get_edge(str(nodes[0]), str(nodes[1]))
  edge.attr['weight'] = w
  edge.attr['label'] = '%.2f' % w

print("Q = ", Q)


# Draw graph
A.layout()
A.draw('file.png')
Image('file.png')



**EXERCISE**: Please correct the chain above to be well behaved.

In [ ]:
# Simple simulation
pi = np.array([0.1, 0.1, 0.1, 0.7])
for i in range(30):
  pi = np.dot(pi, Q)
  print(pi)


# Theoretical Aspects of Constrained Molecular Simulations

Molecules conformations lie in complex highly dimensional spaces, and present a challange for molecular simulations that try to explore them. Take, for example, ethane: although it is a simple molecule, it already has $8 \times 3 = 24$ dimensions. To simplify this endevour some types of degrees of freedom may be constrained: such as the bond lengths. This will increase the integration time step and reduce the number equations solved at each step.





<figure>
<center>
<img src='https://github.com/spirilaurentiu/Notebooks/blob/main/ethane.s.png?raw=true' />
<figcaption>FIGURE 1. Ethane Molecule with or without the bond lengths degrees of freedom taken into account. </figcaption>
</center>
</figure>

## Internal Coordinates
Internal coordinates, also called BAT, are the set of bond lengths, bond angles and bond torsions of a molecular system.


* As opposed to Cartesian coordinates which are absolute, BAT coordinates are relative.
* Allows the external degrees of freedom to be ignored because they are explicitly expressed in BAT
* BAT coordinates have energy terms associated with them which makes trajectories easier to calculate
* Energies associated with the three types of coordinates differ in their order of magnitude which gives them the advantage to easly constrain them by type: eg. one can fix all the bond lengths during molecular dynamics.


$ E(d, \theta, \phi) 
= \displaystyle\sum_{\textbf{B}onds}{\frac{1}{2} k_d (d - d_0)^2} 
+ \displaystyle\sum_{\textbf{A}ngles}{\frac{1}{2} k_a (\theta - \theta_0)^2}
+ \displaystyle\sum_{\textbf{T}orsions}{\frac{1}{2} k_t f(\phi, \phi_0, ...)}
$




## Rigid Body Dynamics 

In the case of biological macromolecules, their particular chemistry may induce certain patterns that can be speculated for the use of constraints. For example, proteins have secondary structures elements such as helices, that can be treated as rigid bodies during some parts of the simulation.

If we apply these type of constraints some molecule parts become rigid and **robot mechanics** algorithms can be employed to solve the equations of motion.
The mechanical part of a robot is defined as a system of articulated bodies, which is a set of bodies (links) connected by articulations (joints).

Constraints can be enforced using:
* maximal coordinates for the links (in which every link is represented with its 6 degrees of freedom) and additional equations for each constraint applyed to the whole system;
* reduced coordinates for articulations degrees of freedom

Robosample uses the complement of the set of constraints: which is the set of the remaining degrees of freedom. These remaining degrees of freedom set is called a "World".




<figure>
<center>
<img src='https://github.com/spirilaurentiu/Notebooks/blob/main/protein.s.png?raw=true' />
<figcaption>FIGURE 2. Proteins secondary structures </figcaption>
</center>
</figure>

Using constraints, by definition, do not take into account all the degrees of freedom therefore suffers from a lack of ergodicity. Also, particular degrees of freedom may be coupled when transitioning from a conformational basin to another. We overcome this in Robosample through the means of "overlaped blocked Gibbs sampling" in a method called GCHMC.

Gibbs sampling is a Markov Chain Monte Carlo algorithm which samples from a multivariate distribution $\pi(X = \{x_i\})$ by taking turns in sampling each of its component $x_i$, conditioned on the previous sampled components $\pi(x_i | \{x_{0,i-1}\})$.

In a similar fashion, the algorithm allows sampling jointly multiple components $x_{i, i+k}$ in what is called "blocked Gibbs sampling". Even more so, the components may overlap to form "overlapped blocked Gibbs sampling".

We will use this particular type of Gibbs sampling in Robosample by alternating multiple definitions of rigid bodies.
 Robosample is a molecular simulation program that uses algorithms designed primarly for robot mechanics. 




**DISSCUTION**

* Why shoud we consider using constraints?
* What macromolecule regions would you pick to constraint during different simulations?


### Further Reading
A comprehensive desciption internal coordinates molecular dynamics and usage of constraints is given in [J Phys Chem B. 2015, **119(4)**, 1233-42](https://pubs.acs.org/doi/10.1021/jp509136y).

The way they are implemented in Robosample GCHMC is found in:  [ J Chem Theory Comput. 2017, **13(10)**, 4649-4659](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5634954/) and [Biochim Biophys Acta Gen Subj. 2020, **1864(8)** 129616](https://pubmed.ncbi.nlm.nih.gov/32298789/).


# Part 0 - Download and Install Robosample
Make sure to enable GPU acceleration 
---



Robosample binaries are already build on Google colab and uploaded on Github. Clone the Github repository that contains Robosample binaries.

In [ ]:
%cd /content/
! if [ -d Notebooks ]; then echo "Notebooks exists"; else git clone --recursive https://github.com/spirilaurentiu/Notebooks.git; fi
!ls Notebooks
%cd Notebooks/
!git pull

 Now we have to make sure that Robosample binary finds its libraries. Set the linker path.
 Robosample relies heavly on GPUs. Set cuda directories.

In [ ]:
%env LD_LIBRARY_PATH=/usr/lib64-nvidia:/usr/local/openmm/lib:/usr/local/openmm/lib/plugins:/content/Notebooks/build/release/robosample/src
!echo "$LD_LIBRARY_PATH"

%env CUDA_INC_DIR=/usr/local/cuda
%env CUDA_ROOT=/usr/local/cuda


%cd /content/Notebooks/build/release/robosample/src/
%mkdir -p /usr/local/openmm/lib/plugins
%cp libOpenMMOpenCL.so libOpenMMRPMDCUDA.so libOpenMMRPMDOpenCL.so libOpenMMCUDA.so libOpenMMCudaCompiler.so libOpenMMCPU.so /usr/local/openmm/lib/plugins/
%cp libOpenMM.so libOpenMM.so.7.5 libOpenMMRPMD.so /usr/local/openmm/lib/

!nvidia-smi

If everything went okay so far, Robosample should run on a test small molecule.

In [ ]:
%cd build/release/robosample/src/
!./GMOLMODEL_robo inp.2but

Now let's install Robosample Python interface. Its Python interface relies on analysis with MDTraj.

In [ ]:
# MDTraj is a prerequisite for robosample.py
!if pip list | grep mdtraj ; then echo "MDTraj is installed"; else pip install mdtraj; fi
!if pip list | grep pytraj ; then echo "PyTraj is installed"; else pip install pytraj; fi
!if pip list | grep nglview ; then echo "NglView is installed"; else pip install -q nglview; fi
!if pip list | grep py3Dmol ; then echo "Py3DMol is installed"; else pip install py3Dmol; fi

# Robosample Python interface is in the tools directory
%env PYTHONPATH=/env/python:/content/Notebooks/tools
!echo $PYTHONPATH


A bit more setup.

In [ ]:

%cd /content/Notebooks/
!cp -i build/release/robosample/src/libOpenMMPlugin.so .
!ls

If everything went OK we should be able to set up a simple example with the Python interface.

In [ ]:
# Run a test script
! if [ -d temp ]; then echo "temp directory exists"; else mkdir -p temp/pdbs; fi
!ls
!python simulate.py

Robosample, by default, writes pdb files as output.

In [ ]:
!ls temp/pdbs

# Part 1 - Prepare a Robosample Script

---

<h1>Why use a script?<!h1>

Molecular dynamics simulations require a lot of information about
* Input data
  * coordinates
  * topology
  * which atoms are included in energy terms
  * parameters for functions in energy terms
* System description
  * periodicity
  * constraints
* Integrators
  * algorithms to propagate forward in time
  * adjust box size
  * adjust kinetic energy (temperature)
* Simulation
  * how long to run
  * how much output data to store

  Let's go through what the parameters mean and I'll give you values for a simple simulation of SARS-CoV MPro C-terminal part (PDB code: 2liz).



<img src='https://github.com/spirilaurentiu/Notebooks/blob/main/mproc.png?raw=true' />

In [ ]:
# Check molecule
!ls mproc/

<h1>Import necessary modules</h1>
First we need to import the module that contains the Robosample interface, robosample.py. Also import here oher modules that we may need.

In [ ]:
# Imports
import os
import sys
if not "/content/Notebooks/tools/" in sys.path:
  sys.path.append("/content/Notebooks/tools/")


# Import Robosample module
from robosample import *


<h1>Load molecular structure files</h1>

* For input files enter “mproc/ligand.prmtop” and “mproc/ligand.min.rst7”
“Platform” describes the version of the code and the hardware it will run on
  * “CUDA” and “OpenCL” are meant for GPUs, which make MD simulations much faster. “CUDA” only works with Nvidia GPUs and “OpenCL” on others
  * “CPU” is a faster version of Reference.

* The properties dictionary indicates the number of threads to be used
* Robosample only supports Amber force field for now.



In [ ]:
# Load Amber files
prmtop = AmberPrmtopFile("mproc/ligand.prmtop")
inpcrd = AmberInpcrdFile("mproc/ligand.min.rst7")

# Hardware platform
platform = Platform.getPlatformByName('GPU')
properties={'nofThreads': 0}


<h1>Create System </h1>

* “Nonbonded method” describes how long-range interactions are treated. The more interactions that need to be computed, the slower an MD simulation will be. 
  * Cutoffs don’t perform calculations if two particles are beyond a certain distance apart.
  * Ewald methods calculate long-range interaction energies between a system and its periodic images. It assumes the system repeats indefinitely.
  * Since we are using implicit solvent, we don’t need periodicity. 
Let’s use “CutoffNonPeriodic”.
* “Constraints”
  * force a degree of freedom to be a certain value

Let’s keep the other “System” parameters as is

In [ ]:
# Remove any existing robots directory
if os.path.exists("robots"):
  shutil.rmtree("robots")

# Create a Robosample system by calling createSystem on prmtop
system = prmtop.createSystem(createDirs = True,
	nonbondedMethod = "CutoffPeriodic",
 	nonbondedCutoff = 1.44*nanometer,
 	constraints = None,
 	rigidWater = True,
 	implicitSolvent = True,
 	soluteDielectric = 1.0,
 	solventDielectric = 78.5,
 	removeCMMotion = False
)


<h1> Choose an Integrator </h1>

* “Integrator” is the algorithm that goes from one configuration to the next
  * Verlet is completely deterministic
  * Langevin adds some random noise to the motion. The level of noise maintains the system at a certain temperature.
  * Brownian is so random that there is no momentum
  * Variable methods use different time steps and depend on an error tolerance
  * Robosample Hybrid Monte Carlo uses Velocity Verlet

<h1> Thermostats </h1>

* allow the kinetic energy of the system to change by modifying velocities
  * Andersen - velocity reinitialization
  * Berendsen - temperature decays to target temperature
  * Nose Hoover - an extra degree of freedom in the Hamiltonian

Robosample uses Andersen

  <h1> Barostats</h1>

* keep the system at a certain pressure 
  * allows the volume of the system to change
    * Hoover
    * Rahman
    * Parrinello

since we are using implicit water, we will not use a barostat.

In [ ]:
# Choose an integrator
integrator = VVIntegrator(300*kelvin,   # Temperature of head bath
                           0.001*picoseconds) # Time step

<h1>Wrap everything in a Simulation Object</h1>

* “Reporters” store data about the simulation

  * “PDB” structure files are saved in the specified output directory
  * “DCD” is a binary file format for molecular dynamics trajectories

* “Report Interval” is how often the data are stored


In [ ]:
# Create Simulation object
simulation = Simulation(prmtop.topology, system, integrator, platform, properties)
simulation.reporters.append(PDBReporter('temp', 3))
simulation.context.setPositions(inpcrd.positions)



<h1> Add Worlds </h1>

The concept of "World" is esential to Robosample.

When simulating the molecular system with constraints, Robosample uses the complement of the set of constraints: which is the set of the remaining degrees of freedom. These remaining degrees of freedom set is called a "World", and corresponds to a Gibbs block.




<h3>Types of Worlds:</h3>

There are different types of world that can be defined by the Robosample Python interface set up by the following keywords:
* *accesibile* : residues chosen based on the solvent accesible surface are (SASA) will give mostly the exposed rotamers - known to give most of, for example, receptor flexibility in ligand binding

* *loops*: regions between secondary structures <br />

* *stretch*: only a particular stretch of amino acids <br />

* *roll*: builds a World for every backbone torsion angle within a stretch <br />


In [ ]:
## Generate NMA-Scaled Flex Files
sAll = [[1, 120]]
nterJoint = [[11, 12]]

simulation.addWorld(regionType='stretch',
                    region=sAll,
                    rootMobility='Weld',
                    timestep=0.003,
                    mdsteps=40,
                    argJointType='Pin',
                    subsets=['rama','side'],
                    samples=1)

simulation.addWorld(regionType='stretch',
                    region=nterJoint,
                    rootMobility='Weld',
                    timestep=0.02,
                    mdsteps=5,
                    argJointType='Pin',
                    subsets=['rama','side'],
                    samples=1)


<h1>Run the simulation </h1>

The Simulation step function in Robosample takes the number of **rounds** of Gibbs sampling as input (not the number of steps of MD).


In [ ]:

# Run simulation
simulation.step(300)



Check the output

In [ ]:
!ls temp/pdbs

A more low-level solution for the step function in the Python interface.

**DON'T RUN THIS IF THE LAST STEP WENT OK**

In [ ]:
# Backup run
!rm temp/pdbs/*
!ls robots/bot0/
!cat inp.test
!./build/release/robosample/src/GMOLMODEL_robo inp.test

In [ ]:
!ls temp/pdbs/* | wc -l

Convert the *pdb* files into a more compact *dcd* trajectory.

In [ ]:
import pytraj as pt
traj = pt.load('temp/pdbs/*.pdb', 'mproc/ligand.prmtop')
pt.write_traj('mproc.dcd', traj, overwrite=True)
!ls -lht

<h2> Viusalize the results </h2>

Install necessary tools

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

try:
  import py3Dmol
except:
  !pip install py3Dmol
  import py3Dmol


Visualize trajectory

In [ ]:
# Visualize
import pytraj as pt
import nglview as nv

traj = pt.load('mproc.dcd', 'mproc/ligand.prmtop')
traj.superpose(ref=0, mask='@CA')
view = nv.show_pytraj(traj)
#view.add_line(traj)
#view.zoomTo()
#view.show()
view

<img src='https://github.com/spirilaurentiu/Notebooks/blob/main/mprocsim.png?raw=true' />

Save the results if you want, although pay attention to the size.

In [ ]:
GitHub_dir = '/content/drive/MyDrive/GitHub'
import os
if not os.path.isdir(GitHub_dir):
  !mkdir -p {GitHub_dir}

os.chdir(GitHub_dir)
if not os.path.isdir(os.path.join(GitHub_dir,'modelingworkshop')):
  !git clone https://github.com/CCBatIIT/modelingworkshop
else:
  os.chdir(os.path.join(GitHub_dir,'modelingworkshop'))
  !git pull origin main